In [1]:
#!/usr/bin/env python

import cv2

import numpy as np

SZ = 20
CLASS_N = 10

# local modules
from common import clock, mosaic

def split2d(img, cell_size, flatten=True):
    h, w = img.shape[:2]
    sx, sy = cell_size
    cells = [np.hsplit(row, w//sx) for row in np.vsplit(img, h//sy)]
    cells = np.array(cells)
    if flatten:
        cells = cells.reshape(-1, sy, sx)
    return cells

def load_digits(fn):
    digits_img = cv2.imread(fn, 0)
    digits = split2d(digits_img, (SZ, SZ))
    labels = np.repeat(np.arange(CLASS_N), len(digits)/CLASS_N)
    return digits, labels

def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img, M, (SZ, SZ), flags=cv2.WARP_INVERSE_MAP | cv2.INTER_LINEAR)
    return img

def svmInit(C=12.5, gamma=0.50625):
    model = cv2.ml.SVM_create()
    model.setGamma(gamma)
    model.setC(C)
    model.setKernel(cv2.ml.SVM_RBF)
    model.setType(cv2.ml.SVM_C_SVC)
  
    return model

def svmTrain(model, samples, responses):
    model.train(samples, cv2.ml.ROW_SAMPLE, responses)
    return model

def svmPredict(model, samples):
    return model.predict(samples)[1].ravel()

def svmEvaluate(model, digits, samples, labels):
    predictions = svmPredict(model, samples)
    print(predictions)
    accuracy = (labels == predictions).mean()
    print('Percentage Accuracy: %.2f %%' % (accuracy*100))

    confusion = np.zeros((10, 10), np.int32)
    for i, j in zip(labels, predictions):
        confusion[int(i), int(j)] += 1
    print('confusion matrix:')
    print(confusion)

    vis = []
    for img, flag in zip(digits, predictions == labels):
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        if not flag:
            img[...,:2] = 0
        
        vis.append(img)
    return mosaic(25, vis)


def preprocess_simple(digits):
    return np.float32(digits).reshape(-1, SZ*SZ) / 255.0


def get_hog() : 
    winSize = (20,20)
    blockSize = (8,8)
    blockStride = (4,4)
    cellSize = (8,8)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    signedGradient = True

    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels, signedGradient)

    return hog
    affine_flags = cv2.WARP_INVERSE_MAP|cv2.INTER_LINEAR



In [2]:
def get_hog_descriptors(digits_deskewed):
    print('Defining HoG parameters ...')
    # HoG feature descriptor
    hog = get_hog();

    print('Calculating HoG descriptor for every image ... ')
    hog_descriptors = []
    for img in digits_deskewed:
        hog_descriptors.append(hog.compute(img))
    hog_descriptors = np.squeeze(hog_descriptors)
    return hog_descriptors


In [3]:
#if __name__ == '__main__':
print('Loading digits from digits.png ... ')
# Load data.
digits, labels = load_digits('digits.png')
print("digits.shape", digits.shape)

print('Shuffle data ... ')
# Shuffle data
rand = np.random.RandomState(10)
shuffle = rand.permutation(len(digits))
digits, labels = digits[shuffle], labels[shuffle]

print('Deskew images ... ')
digits_deskewed = list(map(deskew, digits))

hog_descriptors = get_hog_descriptors(digits_deskewed)
print('hog_descriptors.shape', hog_descriptors.shape)

print('Spliting data into training (90%) and test set (10%)... ')
train_n=int(0.9*len(hog_descriptors))
digits_train, digits_test = np.split(digits_deskewed, [train_n])
hog_descriptors_train, hog_descriptors_test = np.split(hog_descriptors, [train_n])
labels_train, labels_test = np.split(labels, [train_n])


print('Training SVM model ...')
model = svmInit()
svmTrain(model, hog_descriptors_train, labels_train)


Loading digits from digits.png ... 
digits.shape (5000, 20, 20)
Shuffle data ... 
Deskew images ... 
Defining HoG parameters ...
Calculating HoG descriptor for every image ... 
hog_descriptors.shape (5000, 144)
Spliting data into training (90%) and test set (10%)... 
Training SVM model ...


<ml_SVM 0000017BEB0C8B10>

In [4]:
digits_test.shape

(500, 20, 20)

In [5]:
hog_descriptors_test.shape

(500, 144)

In [6]:
def digits_to_hog(digits):
    deskewed_digits = list(map(deskew, digits))
    return get_hog_descriptors(deskewed_digits)

def predict_digit(model, digit):
    return svmPredict(model, digits_to_hog(digit.reshape(1, 20, 20)))
    
def predict_digits(model, digits):
    return svmPredict(model, digits_to_hog(digits))

In [7]:
#20, 800 => 8
print('Evaluating model ... ')
num_tests = 2
#vis = svmEvaluate(model, digits_test[0:num_tests], hog_descriptors_test[0:num_tests], labels_test[0:num_tests])
# predictions = svmPredict(model, hog_descriptors_test[0:num_tests])

print('SHHHAPPPEEEE', digits_test[0:num_tests].shape)
#print('SHHHAPPPEEEE2', digits_test[0].reshape(1,20,20).shape)
#predictions = svmPredict(model, digits_to_hog(digits_test[0:num_tests]))
# prediction = predict_digit(model, digits_test[0])

predictions = predict_digits(model, digits_test[0:num_tests])

print('predictions', predictions)
# cv2.imwrite("digits-classification.jpg",vis)

Evaluating model ... 
SHHHAPPPEEEE (2, 20, 20)
Defining HoG parameters ...
Calculating HoG descriptor for every image ... 
predictions [3. 9.]


In [8]:
vis.shape

NameError: name 'vis' is not defined

In [72]:
# 768141
image = cv2.imread('722281.jpg')
print(image.shape)
sqsize = 57

sq1_sx = 722 + sqsize * 2
sq1_sy = 446

sq1 = image[sq1_sy:(sq1_sy + sqsize), sq1_sx:(sq1_sx + sqsize)]

#cv2.rectangle(image, (sq1_sx, sq1_sy), (sq1_sx + sqsize, sq1_sy + sqsize), (255,0,0), 2)
print('')
#cv2.rectangle

#cv2.resize(img, (width, height))

#cv2.imshow("test", image)
#cv2.waitKey(0)
#cv2.imshow("test", sq1)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


(1643, 2555, 3)



In [73]:
#sq1 = cv2.resize(sq1, (20, 20))
sq1 = cv2.cvtColor(sq1, cv2.COLOR_BGR2GRAY)
_,sq1 = cv2.threshold(sq1,127,255,cv2.THRESH_BINARY_INV)
kernel = np.ones((5,5),np.uint8)
sq1 = cv2.dilate(sq1,kernel,iterations = 1)
#_,sq1 = cv2.threshold(sq1,150,255,cv2.THRESH_BINARY_INV)
#cv2.bitwise_not(img)

cv2.imshow("test", cv2.resize(sq1, (200, 200)))
cv2.waitKey(0)
cv2.destroyAllWindows()

sq1 = cv2.resize(sq1, (20, 20))
my_digits = np.array([sq1, sq1])
my_digits.shape

predictions = predict_digits(model, my_digits)
print(predictions)

Defining HoG parameters ...
Calculating HoG descriptor for every image ... 
[4. 4.]
